
# Data Analysis for cost of care in healthcare
_by Hafsa Laeeque_

---
The task is to<br>
  > 1) analyze the **clinical and financial data** of patients hospitalized for a _certain condition_.<br>
    2) join data given in the different tables.<br>
    3) find insights about **drivers of cost of care**.<br>
    4) document _approach, results and insights_ using [slides](https://docs.google.com/presentation/d/1-gYni51iGkYh4OCCr-BYKAV3YqUH4OzQOUAtz5MkrEg/edit?usp=sharing) and a [document](https://docs.google.com/document/d/1fQB0AP2ue_zKVUUAHx_1sJ626okOMtm_IdLOnw-9__A/edit?usp=sharing), both of which should have a similar narrative.<br>

---
There are 4 excel files in the [datasets](https://github.com/hafsalaeeque/cost-in-healthcare-DS-proj/tree/master/datasets) folder which I will be using.

## Analysing the datasets
### Import packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime, timedelta 
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
#display columns and rows
pd.options.display.max_columns = 50
pd.options.display.max_rows = 4000

### (A) Load the datasets
A1 - Let's load the first dataset `bill_amount.csv`.

In [3]:
bill_amt = pd.read_csv('datasets/bill_amount.csv')

In [4]:
bill_amt.shape

(13600, 2)

In [5]:
bill_amt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13600 entries, 0 to 13599
Data columns (total 2 columns):
bill_id    13600 non-null int64
amount     13600 non-null float64
dtypes: float64(1), int64(1)
memory usage: 212.6 KB


In [6]:
bill_amt.head()

,bill_id,amount
0,40315104,1552.634830
1,2660045161,1032.011951
2,1148334643,6469.605351
3,3818426276,755.965425
4,9833541918,897.347816


In [7]:
bill_amt.bill_id.nunique()

13600

The first excel file, `bill_amt.csv`, does not have any null values. The data shows the cost incurred for each unique bill.

---
A2 - Let's load the second dataset `bill_id.csv`.

In [8]:
bill_id_df = pd.read_csv('datasets/bill_id.csv')

In [9]:
bill_id_df.shape

(13600, 3)

In [10]:
bill_id_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13600 entries, 0 to 13599
Data columns (total 3 columns):
bill_id              13600 non-null int64
patient_id           13600 non-null object
date_of_admission    13600 non-null object
dtypes: int64(1), object(2)
memory usage: 318.8+ KB


In [11]:
bill_id_df.tail()

,bill_id,patient_id,date_of_admission
13595,1641053864,a4c61deaa9ce86b4d2289eab6128b872,2015-12-28
13596,6956955826,ac52a32f8ce8c46d82df2d72052ae5a9,2015-12-28
13597,1399259594,4f67a54ab205cc9e7e2b0a4ee08e4fba,2015-12-28
13598,9243628699,a4c61deaa9ce86b4d2289eab6128b872,2015-12-28
13599,4808173213,a4c61deaa9ce86b4d2289eab6128b872,2015-12-28


In [12]:
bill_id_df.bill_id.nunique()

13600

The second excel file, `bill_id.csv`, does not have any null values. The data shows the _date_ for when a patient was admitted in the hospital for each unique bill. This could be joined with our first dataset so we can observe the cost incurred for each patient.

---
A3 - Let's load the third dataset `clinical_data.csv`.

In [13]:
data = pd.read_csv('datasets/clinical_data.csv')

In [14]:
data.shape

(3400, 26)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3400 entries, 0 to 3399
Data columns (total 26 columns):
id                    3400 non-null object
date_of_admission     3400 non-null object
date_of_discharge     3400 non-null object
medical_history_1     3400 non-null int64
medical_history_2     3167 non-null float64
medical_history_3     3400 non-null object
medical_history_4     3400 non-null int64
medical_history_5     3096 non-null float64
medical_history_6     3400 non-null int64
medical_history_7     3400 non-null int64
preop_medication_1    3400 non-null int64
preop_medication_2    3400 non-null int64
preop_medication_3    3400 non-null int64
preop_medication_4    3400 non-null int64
preop_medication_5    3400 non-null int64
preop_medication_6    3400 non-null int64
symptom_1             3400 non-null int64
symptom_2             3400 non-null int64
symptom_3             3400 non-null int64
symptom_4             3400 non-null int64
symptom_5             3400 non-null int64
lab

In [16]:
data.isnull().sum().sum()

537

In [17]:
data.head(6)

,id,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,preop_medication_1,preop_medication_2,preop_medication_3,preop_medication_4,preop_medication_5,preop_medication_6,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height
0,1d21f2be18683991eb93d182d6b2d220,2011-01-01,2011-01-11,0,1.0,0,0,0.0,0,0,1,0,1,0,0,1,0,0,0,1,1,13.2,30.9,123.0,71.3,161.0
1,62bdca0b95d97e99e1c712048fb9fd09,2011-01-01,2011-01-11,0,0.0,0,0,0.0,0,0,0,1,1,1,1,0,0,0,1,1,1,13.8,22.6,89.0,78.4,160.0
2,c85cf97bc6307ded0dd4fef8bad2fa09,2011-01-02,2011-01-13,0,0.0,0,0,0.0,0,0,0,1,1,1,1,1,1,1,1,1,0,11.2,26.2,100.0,72.0,151.0
3,e0397dd72caf4552c5babebd3d61736c,2011-01-02,2011-01-14,0,1.0,No,0,0.0,1,1,1,0,1,0,0,1,1,1,1,1,1,13.3,28.4,76.0,64.4,152.0
4,94ade3cd5f66f4584902554dff170a29,2011-01-08,2011-01-16,0,0.0,No,0,0.0,1,1,0,0,0,0,1,0,0,1,0,1,0,12.0,27.8,87.0,55.6,160.0
5,59e07adc2dbc5f70131f57d003610d74,2011-01-07,2011-01-17,0,NaN,No,0,NaN,0,0,0,1,1,1,1,1,1,0,1,1,1,15.8,31.0,75.0,78.8,169.0


The third excel file, `clinical_data.csv` has 25 types of clinical data entries of 3400 patients.<br>
- It covers the
    - the date as to when the patient was admitted and discharged,
    - height of patient,
    - weight of patient,
    - if patient has any/all 7 medical histories, 
    - if patient has had to take any/all the 6 preoperation medication, 
    - if patient showed any/all the 5 symptoms and
    - 3 lab results of patient<br>
    
- The `id` column of the dataset seems to be the `patient_id` column in the `bill_id_df` datset.<br>
- It has **537 null values** in the 2nd and 5th medical history data.<br>
- The dataset needs to be cleaned as there are
     - string objects like "No" in the 3rd medical history column and<br>
     - null values.<br>
     
Considering that we had 13,600 unique patients in the `bill_id_df` dataset, the `clinical_data.csv` has only 25% of these patients' clinical data. 

---
A4 - Let's load the last dataset `demographics.csv`.

In [18]:
demographic = pd.read_csv('datasets/demographics.csv')

In [19]:
demographic.shape

(3000, 5)

In [20]:
demographic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 5 columns):
patient_id         3000 non-null object
gender             3000 non-null object
race               3000 non-null object
resident_status    3000 non-null object
date_of_birth      3000 non-null object
dtypes: object(5)
memory usage: 117.3+ KB


In [21]:
demographic.head(10)

,patient_id,gender,race,resident_status,date_of_birth
0,fa2d818b2261e44e30628ad1ac9cc72c,Female,Indian,Singaporean,1971-05-14
1,5b6477c5de78d0b138e3b0c18e21d0ae,f,Chinese,Singapore citizen,1976-02-18
2,320aa16c61937447fd6631bf635e7fde,Male,Chinese,Singapore citizen,1982-07-03
3,c7f3881684045e6c49020481020fae36,Male,Malay,Singapore citizen,1947-06-15
4,541ad077cb4a0e64cc422673afe28aef,m,Chinese,Singaporean,1970-12-12
5,cf280265a73331d6cad35b4800e96abf,Female,Chinese,PR,1966-12-01
6,94f7d3a8a4d6bb14859b64c3f03e4a6c,m,Malay,Singaporean,1975-09-14
7,43dfbeb8d76f3b00b8fa7a49e5a3eb6f,f,chinese,Singaporean,1974-03-04
8,2882e70ff56c2600bbbbb855fcfa96b9,Male,Chinese,Singaporean,1969-04-22
9,36e65f14c328fef0b02aa7d4047c6f74,Female,Chinese,Singapore citizen,1976-10-24


The last excel file, `demographics.csv` has 4 categories of 3000 patients that tell us more about the patient.<br>
- It has the
    - gender of patient,
    - race of patient,
    - resident status of patient and
    - date of birth of patient<br>
    
- The `patient_id` column of the dataset seems to be the `patient_id` column in the `bill_id_df` datset.<br>
- There are no null values.<br>
- The dataset needs to be cleaned as there are
     - dates in object type<br>
     - synonymous terms representing gender, race & citizenship.<br>
     
Considering the `clinical_data.csv` had 3400 patients' clinical data, the `demographics.csv` only has demogrpahic data for 88% of these patients. 

---
I will clean the datasets for our analysis later on.

### (B) Clean the datasets
`bill_amt` dataset does not require any cleaning.<br>
`bill_id_df` dataset column `date_of_admission` needs to be changed to datetime object.<br>
`data` dataset column `medical_history_3` has string objects that needs to be changed to binary & dates to be changed to datetime objects.<br>
`demographic` dataset column for gender, race and citizenship needs to be changed for consistency & dates to be changed to datetime objects.<br>

---
B1 - Clean `bill_id_df`

In [22]:
bill_id_df['date_of_admission'] = pd.to_datetime(bill_id_df.date_of_admission)

In [23]:
bill_id_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13600 entries, 0 to 13599
Data columns (total 3 columns):
bill_id              13600 non-null int64
patient_id           13600 non-null object
date_of_admission    13600 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 318.8+ KB



---
B2 - Clean `data`

In [24]:
data['date_of_admission'] = pd.to_datetime(data.date_of_admission)

In [25]:
data['date_of_discharge'] = pd.to_datetime(data.date_of_discharge)

In [26]:
data.medical_history_3.unique()

array(['0', 'No', '1', 'Yes'], dtype=object)

In [27]:
data.medical_history_3.value_counts()

0      2176
No      761
1       348
Yes     115
Name: medical_history_3, dtype: int64

In [28]:
data[data['medical_history_3'] == 'No'].head()

,id,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,preop_medication_1,preop_medication_2,preop_medication_3,preop_medication_4,preop_medication_5,preop_medication_6,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height
3,e0397dd72caf4552c5babebd3d61736c,2011-01-02,2011-01-14,0,1.0,No,0,0.0,1,1,1,0,1,0,0,1,1,1,1,1,1,13.3,28.4,76.0,64.4,152.0
4,94ade3cd5f66f4584902554dff170a29,2011-01-08,2011-01-16,0,0.0,No,0,0.0,1,1,0,0,0,0,1,0,0,1,0,1,0,12.0,27.8,87.0,55.6,160.0
5,59e07adc2dbc5f70131f57d003610d74,2011-01-07,2011-01-17,0,NaN,No,0,NaN,0,0,0,1,1,1,1,1,1,0,1,1,1,15.8,31.0,75.0,78.8,169.0
8,457402b26562d41f4e40906d3d17d5d1,2011-01-12,2011-01-18,0,0.0,No,0,0.0,0,0,0,1,1,1,1,0,1,1,1,0,0,12.5,32.9,87.0,98.4,166.0
10,d45fcf3bec63ae3b436be42fad33b9e0,2011-01-12,2011-01-18,1,NaN,No,1,0.0,0,0,0,1,1,1,1,1,1,1,0,1,0,17.0,27.7,116.0,91.6,169.0


In [29]:
data['medical_history_3'] = data['medical_history_3'].map(lambda x: 0.0 if x.strip() == 'No' else 1.0 if x == 'Yes' else float(x.strip()))

In [30]:
data.medical_history_3.value_counts()

0.0    2937
1.0     463
Name: medical_history_3, dtype: int64

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3400 entries, 0 to 3399
Data columns (total 26 columns):
id                    3400 non-null object
date_of_admission     3400 non-null datetime64[ns]
date_of_discharge     3400 non-null datetime64[ns]
medical_history_1     3400 non-null int64
medical_history_2     3167 non-null float64
medical_history_3     3400 non-null float64
medical_history_4     3400 non-null int64
medical_history_5     3096 non-null float64
medical_history_6     3400 non-null int64
medical_history_7     3400 non-null int64
preop_medication_1    3400 non-null int64
preop_medication_2    3400 non-null int64
preop_medication_3    3400 non-null int64
preop_medication_4    3400 non-null int64
preop_medication_5    3400 non-null int64
preop_medication_6    3400 non-null int64
symptom_1             3400 non-null int64
symptom_2             3400 non-null int64
symptom_3             3400 non-null int64
symptom_4             3400 non-null int64
symptom_5             3400 n

In [32]:
data.head()

,id,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,preop_medication_1,preop_medication_2,preop_medication_3,preop_medication_4,preop_medication_5,preop_medication_6,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height
0,1d21f2be18683991eb93d182d6b2d220,2011-01-01,2011-01-11,0,1.0,0.0,0,0.0,0,0,1,0,1,0,0,1,0,0,0,1,1,13.2,30.9,123.0,71.3,161.0
1,62bdca0b95d97e99e1c712048fb9fd09,2011-01-01,2011-01-11,0,0.0,0.0,0,0.0,0,0,0,1,1,1,1,0,0,0,1,1,1,13.8,22.6,89.0,78.4,160.0
2,c85cf97bc6307ded0dd4fef8bad2fa09,2011-01-02,2011-01-13,0,0.0,0.0,0,0.0,0,0,0,1,1,1,1,1,1,1,1,1,0,11.2,26.2,100.0,72.0,151.0
3,e0397dd72caf4552c5babebd3d61736c,2011-01-02,2011-01-14,0,1.0,0.0,0,0.0,1,1,1,0,1,0,0,1,1,1,1,1,1,13.3,28.4,76.0,64.4,152.0
4,94ade3cd5f66f4584902554dff170a29,2011-01-08,2011-01-16,0,0.0,0.0,0,0.0,1,1,0,0,0,0,1,0,0,1,0,1,0,12.0,27.8,87.0,55.6,160.0


In [33]:
data.rename(index=str, columns={"id": "patient_id"}, inplace= True)

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3400 entries, 0 to 3399
Data columns (total 26 columns):
patient_id            3400 non-null object
date_of_admission     3400 non-null datetime64[ns]
date_of_discharge     3400 non-null datetime64[ns]
medical_history_1     3400 non-null int64
medical_history_2     3167 non-null float64
medical_history_3     3400 non-null float64
medical_history_4     3400 non-null int64
medical_history_5     3096 non-null float64
medical_history_6     3400 non-null int64
medical_history_7     3400 non-null int64
preop_medication_1    3400 non-null int64
preop_medication_2    3400 non-null int64
preop_medication_3    3400 non-null int64
preop_medication_4    3400 non-null int64
preop_medication_5    3400 non-null int64
preop_medication_6    3400 non-null int64
symptom_1             3400 non-null int64
symptom_2             3400 non-null int64
symptom_3             3400 non-null int64
symptom_4             3400 non-null int64
symptom_5             3400 non-nu


---
B3 - Clean `demographic`

In [35]:
demographic['date_of_birth'] = pd.to_datetime(demographic.date_of_birth)

In [36]:
demographic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 5 columns):
patient_id         3000 non-null object
gender             3000 non-null object
race               3000 non-null object
resident_status    3000 non-null object
date_of_birth      3000 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 117.3+ KB


In [37]:
demographic.race.unique()

array(['Indian', 'Chinese', 'Malay', 'chinese', 'India', 'Others'],
      dtype=object)

In [38]:
demographic.patient_id.nunique()

3000

In [39]:
demographic.race.value_counts()

Chinese    1608
Malay       629
chinese     307
Indian      195
Others      161
India       100
Name: race, dtype: int64

In [40]:
demographic['race'] = demographic['race'].map(lambda x: 'Chinese' if x.strip() == 'chinese' else 'Indian' if x == 'India' else str(x.strip()))

In [41]:
demographic.race.value_counts()

Chinese    1915
Malay       629
Indian      295
Others      161
Name: race, dtype: int64

In [42]:
demographic.resident_status.unique()

array(['Singaporean', 'Singapore citizen', 'PR', 'Foreigner'],
      dtype=object)

In [43]:
demographic.resident_status.value_counts()

Singaporean          1782
Singapore citizen     610
PR                    465
Foreigner             143
Name: resident_status, dtype: int64

In [44]:
demographic['resident_status'] = demographic['resident_status'].map(lambda x: 'Singaporean' if x.strip() == 'Singapore citizen' else str(x.strip()))

In [45]:
demographic.resident_status.value_counts()

Singaporean    2392
PR              465
Foreigner       143
Name: resident_status, dtype: int64

In [46]:
demographic.gender.unique()

array(['Female', 'f', 'Male', 'm'], dtype=object)

In [47]:
demographic.gender.value_counts()

Female    1396
Male      1333
m          170
f          101
Name: gender, dtype: int64

In [48]:
demographic['gender'] = demographic['gender'].map(lambda x: 'Female' if x.strip() == 'f' else 'Male' if x == 'm' else str(x.strip()))

In [49]:
demographic.gender.value_counts()

Male      1503
Female    1497
Name: gender, dtype: int64

---
B4 - Save the cleaned in data to the cleaned_dataset folder

In [50]:
# bill_id_df.to_csv('clean_datasets/bill_id.csv', sep=',')

In [51]:
# data.to_csv('clean_datasets/clinical_data.csv', sep=',')

In [52]:
# demographic.to_csv('clean_datasets/demographics.csv', sep=',')

### (C) Join the datasets

From the problem at hand, we know that we are analysing insights of the factors that drives cost for patients sharing the same condition. The patients' personal details, medical bill and report data has been provided. As cost is the main focus, we need to combine the dataset so that it shows all the different features that results in the cost.
- Combine `bill_id_df` to `bill_amt`, using the bill_id column & save it as bill data,
- Combine `demographic` to `data`, using the id column to patient_id column & save it as patients' data,
- Combine patients' data to bill, using the id column & save it as patients' cost data.

---
C1 - Join `bill_id_df` & `bill_amt`

In [53]:
bill = pd.merge(bill_id_df, bill_amt, on= 'bill_id')

In [54]:
bill.head()

,bill_id,patient_id,date_of_admission,amount
0,7968360812,1d21f2be18683991eb93d182d6b2d220,2011-01-01,951.214467
1,6180579974,62bdca0b95d97e99e1c712048fb9fd09,2011-01-01,19900.297086
2,7512568183,1d21f2be18683991eb93d182d6b2d220,2011-01-01,12338.269972
3,3762633379,62bdca0b95d97e99e1c712048fb9fd09,2011-01-01,1145.372193
4,7654730355,1d21f2be18683991eb93d182d6b2d220,2011-01-01,1133.601902


In [55]:
bill[bill['patient_id'] == '4e46fddfa404b306809c350aecbf0f6a']

,bill_id,patient_id,date_of_admission,amount
2517,839597397,4e46fddfa404b306809c350aecbf0f6a,2011-11-23,6132.750887
2519,1658432711,4e46fddfa404b306809c350aecbf0f6a,2011-11-23,868.435533
2520,1808393173,4e46fddfa404b306809c350aecbf0f6a,2011-11-23,10361.862067
2523,9525586787,4e46fddfa404b306809c350aecbf0f6a,2011-11-23,1354.232142
5433,543243535,4e46fddfa404b306809c350aecbf0f6a,2013-01-06,769.098299
5435,1288390292,4e46fddfa404b306809c350aecbf0f6a,2013-01-06,7675.021417
5436,9600908611,4e46fddfa404b306809c350aecbf0f6a,2013-01-06,1269.881609
5437,7875201153,4e46fddfa404b306809c350aecbf0f6a,2013-01-06,797.989644
5564,118453328,4e46fddfa404b306809c350aecbf0f6a,2013-01-23,1557.500102
5567,3680165171,4e46fddfa404b306809c350aecbf0f6a,2013-01-23,592.684082


In [56]:
bill = bill.groupby(['patient_id', 'date_of_admission']).sum().sum(
    level=['patient_id', 'date_of_admission']).drop(['bill_id'], axis = 1).reset_index()


In [57]:
bill[bill['patient_id'] == '4e46fddfa404b306809c350aecbf0f6a']

,patient_id,date_of_admission,amount
1024,4e46fddfa404b306809c350aecbf0f6a,2011-11-23,18717.280628
1025,4e46fddfa404b306809c350aecbf0f6a,2013-01-06,10511.990970
1026,4e46fddfa404b306809c350aecbf0f6a,2013-01-23,4949.806331
1027,4e46fddfa404b306809c350aecbf0f6a,2013-09-16,28028.320179


In [89]:
bill_per_patient = bill.groupby(['patient_id']).sum().sum(
    level=['patient_id']).reset_index()

In [90]:
bill_per_patient[bill_per_patient['patient_id'] == '4e46fddfa404b306809c350aecbf0f6a']

,patient_id,amount
921,4e46fddfa404b306809c350aecbf0f6a,62207.398108


In [91]:
bill_per_patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
patient_id    3000 non-null object
amount        3000 non-null float64
dtypes: float64(1), object(1)
memory usage: 46.9+ KB


In [59]:
# bill.to_csv('clean_datasets/bill.csv', sep=',')

In [92]:
bill_per_patient = pd.merge(demographic, bill_per_patient, on= ['patient_id'])

In [94]:
now = datetime.now()
bill_per_patient['age'] = (now - bill_per_patient.date_of_birth).astype('<m8[Y]')

In [95]:
bill_per_patient[bill_per_patient['patient_id'] == '4e46fddfa404b306809c350aecbf0f6a']

,patient_id,gender,race,resident_status,date_of_birth,amount,age
985,4e46fddfa404b306809c350aecbf0f6a,Male,Chinese,Singaporean,1948-06-16,62207.398108,70.0


In [96]:
# bill_per_patient.to_csv('clean_datasets/bill_per_patient.csv', sep=',')

---
C2 - Join `demographic` & `data`

In [60]:
patient_data = pd.merge(demographic, data, on= 'patient_id')

In [61]:
patient_data.head()

,patient_id,gender,race,resident_status,date_of_birth,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,preop_medication_1,preop_medication_2,preop_medication_3,preop_medication_4,preop_medication_5,preop_medication_6,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height
0,fa2d818b2261e44e30628ad1ac9cc72c,Female,Indian,Singaporean,1971-05-14,2014-09-20,2014-10-04,0,0.0,0.0,0,0.0,0,1,1,0,1,0,1,1,1,1,0,0,1,14.2,27.0,81.0,68.1,161.0
1,5b6477c5de78d0b138e3b0c18e21d0ae,Female,Chinese,Singaporean,1976-02-18,2014-08-08,2014-08-18,0,0.0,0.0,0,NaN,0,1,0,0,0,1,1,1,1,1,1,1,1,14.3,27.2,92.0,76.5,151.0
2,320aa16c61937447fd6631bf635e7fde,Male,Chinese,Singaporean,1982-07-03,2011-03-10,2011-03-20,0,0.0,0.0,0,0.0,1,0,0,1,1,1,1,0,0,0,1,1,1,14.4,26.6,92.0,69.7,175.0
3,320aa16c61937447fd6631bf635e7fde,Male,Chinese,Singaporean,1982-07-03,2013-07-02,2013-07-14,0,0.0,0.0,1,NaN,0,0,1,1,0,1,0,1,1,0,0,1,0,18.0,26.6,92.0,66.7,175.0
4,c7f3881684045e6c49020481020fae36,Male,Malay,Singaporean,1947-06-15,2015-10-28,2015-11-10,0,1.0,0.0,0,0.0,0,0,0,1,0,0,1,1,1,1,1,1,1,14.9,26.7,97.0,96.9,180.0


In [62]:
patient_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3400 entries, 0 to 3399
Data columns (total 30 columns):
patient_id            3400 non-null object
gender                3400 non-null object
race                  3400 non-null object
resident_status       3400 non-null object
date_of_birth         3400 non-null datetime64[ns]
date_of_admission     3400 non-null datetime64[ns]
date_of_discharge     3400 non-null datetime64[ns]
medical_history_1     3400 non-null int64
medical_history_2     3167 non-null float64
medical_history_3     3400 non-null float64
medical_history_4     3400 non-null int64
medical_history_5     3096 non-null float64
medical_history_6     3400 non-null int64
medical_history_7     3400 non-null int64
preop_medication_1    3400 non-null int64
preop_medication_2    3400 non-null int64
preop_medication_3    3400 non-null int64
preop_medication_4    3400 non-null int64
preop_medication_5    3400 non-null int64
preop_medication_6    3400 non-null int64
symptom_1       

In [81]:
patient_data.patient_id.value_counts().head()

4e46fddfa404b306809c350aecbf0f6a    4
b2d15cda8c4e1f86ba43356434df6718    3
f01974d944d1c4b2c40753223fb979fa    3
e3270f450ee164e596ca933a25bab61d    3
d15ec3b278e42b36e7362ac783643685    3
Name: patient_id, dtype: int64

`patient_data` has data of 3000 unique patients, with patients who have been admitted for 1 to 4 times. Only 1 patient has been admitted for 4 times and most have been admitted only once.<br>
<br>
Let's add the frequency of being admitted in the dataset.

In [64]:
patient_data['freq']= patient_data.groupby('patient_id')['patient_id'].transform('count')

In [65]:
patient_data.head()

,patient_id,gender,race,resident_status,date_of_birth,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,preop_medication_1,preop_medication_2,preop_medication_3,preop_medication_4,preop_medication_5,preop_medication_6,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height,freq
0,fa2d818b2261e44e30628ad1ac9cc72c,Female,Indian,Singaporean,1971-05-14,2014-09-20,2014-10-04,0,0.0,0.0,0,0.0,0,1,1,0,1,0,1,1,1,1,0,0,1,14.2,27.0,81.0,68.1,161.0,1
1,5b6477c5de78d0b138e3b0c18e21d0ae,Female,Chinese,Singaporean,1976-02-18,2014-08-08,2014-08-18,0,0.0,0.0,0,NaN,0,1,0,0,0,1,1,1,1,1,1,1,1,14.3,27.2,92.0,76.5,151.0,1
2,320aa16c61937447fd6631bf635e7fde,Male,Chinese,Singaporean,1982-07-03,2011-03-10,2011-03-20,0,0.0,0.0,0,0.0,1,0,0,1,1,1,1,0,0,0,1,1,1,14.4,26.6,92.0,69.7,175.0,2
3,320aa16c61937447fd6631bf635e7fde,Male,Chinese,Singaporean,1982-07-03,2013-07-02,2013-07-14,0,0.0,0.0,1,NaN,0,0,1,1,0,1,0,1,1,0,0,1,0,18.0,26.6,92.0,66.7,175.0,2
4,c7f3881684045e6c49020481020fae36,Male,Malay,Singaporean,1947-06-15,2015-10-28,2015-11-10,0,1.0,0.0,0,0.0,0,0,0,1,0,0,1,1,1,1,1,1,1,14.9,26.7,97.0,96.9,180.0,1


In [66]:
patient_data[patient_data['patient_id'] == '4e46fddfa404b306809c350aecbf0f6a']

,patient_id,gender,race,resident_status,date_of_birth,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,preop_medication_1,preop_medication_2,preop_medication_3,preop_medication_4,preop_medication_5,preop_medication_6,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height,freq
1113,4e46fddfa404b306809c350aecbf0f6a,Male,Chinese,Singaporean,1948-06-16,2011-11-23,2011-12-05,1,1.0,0.0,0,0.0,0,0,1,1,1,0,0,1,1,0,1,1,0,16.1,25.5,111.0,71.4,166.0,4
1114,4e46fddfa404b306809c350aecbf0f6a,Male,Chinese,Singaporean,1948-06-16,2013-01-06,2013-01-17,0,0.0,0.0,0,0.0,0,0,0,1,1,1,1,1,1,0,1,0,0,14.1,24.1,103.0,71.4,166.0,4
1115,4e46fddfa404b306809c350aecbf0f6a,Male,Chinese,Singaporean,1948-06-16,2013-01-23,2013-02-02,0,0.0,0.0,0,0.0,0,1,0,0,1,1,0,1,0,0,0,0,0,13.0,27.0,91.0,70.4,166.0,4
1116,4e46fddfa404b306809c350aecbf0f6a,Male,Chinese,Singaporean,1948-06-16,2013-09-16,2013-09-27,1,1.0,0.0,0,0.0,1,0,0,1,1,0,1,0,1,1,0,0,1,11.7,22.6,84.0,70.4,166.0,4


In [67]:
patient_data['freq'].value_counts()

1    2621
2     718
3      57
4       4
Name: freq, dtype: int64

Most have been hospitalized for 1 day only.

---

In [68]:
patient_data['days_admitted'] = patient_data.date_of_discharge - patient_data.date_of_admission

In [69]:
patient_data.head()

,patient_id,gender,race,resident_status,date_of_birth,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,preop_medication_1,preop_medication_2,preop_medication_3,preop_medication_4,preop_medication_5,preop_medication_6,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height,freq,days_admitted
0,fa2d818b2261e44e30628ad1ac9cc72c,Female,Indian,Singaporean,1971-05-14,2014-09-20,2014-10-04,0,0.0,0.0,0,0.0,0,1,1,0,1,0,1,1,1,1,0,0,1,14.2,27.0,81.0,68.1,161.0,1,14 days
1,5b6477c5de78d0b138e3b0c18e21d0ae,Female,Chinese,Singaporean,1976-02-18,2014-08-08,2014-08-18,0,0.0,0.0,0,NaN,0,1,0,0,0,1,1,1,1,1,1,1,1,14.3,27.2,92.0,76.5,151.0,1,10 days
2,320aa16c61937447fd6631bf635e7fde,Male,Chinese,Singaporean,1982-07-03,2011-03-10,2011-03-20,0,0.0,0.0,0,0.0,1,0,0,1,1,1,1,0,0,0,1,1,1,14.4,26.6,92.0,69.7,175.0,2,10 days
3,320aa16c61937447fd6631bf635e7fde,Male,Chinese,Singaporean,1982-07-03,2013-07-02,2013-07-14,0,0.0,0.0,1,NaN,0,0,1,1,0,1,0,1,1,0,0,1,0,18.0,26.6,92.0,66.7,175.0,2,12 days
4,c7f3881684045e6c49020481020fae36,Male,Malay,Singaporean,1947-06-15,2015-10-28,2015-11-10,0,1.0,0.0,0,0.0,0,0,0,1,0,0,1,1,1,1,1,1,1,14.9,26.7,97.0,96.9,180.0,1,13 days


In [70]:
patient_data['days_admitted'].value_counts()

11 days    437
12 days    414
10 days    412
9 days     402
13 days    389
8 days     309
14 days    269
15 days    227
7 days     166
16 days    113
6 days     111
17 days     52
5 days      44
18 days     21
4 days      15
3 days       8
19 days      6
2 days       2
20 days      2
1 days       1
Name: days_admitted, dtype: int64

Most get hospitalized for 9 to 12 days.

---
Let's assume all our patients are living and none of them have passed on. We will add a column to represent their age.

In [71]:
now = datetime.now()
patient_data['age'] = (now - patient_data.date_of_birth).astype('<m8[Y]')

In [72]:
patient_data.head()

,patient_id,gender,race,resident_status,date_of_birth,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,preop_medication_1,preop_medication_2,preop_medication_3,preop_medication_4,preop_medication_5,preop_medication_6,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height,freq,days_admitted,age
0,fa2d818b2261e44e30628ad1ac9cc72c,Female,Indian,Singaporean,1971-05-14,2014-09-20,2014-10-04,0,0.0,0.0,0,0.0,0,1,1,0,1,0,1,1,1,1,0,0,1,14.2,27.0,81.0,68.1,161.0,1,14 days,47.0
1,5b6477c5de78d0b138e3b0c18e21d0ae,Female,Chinese,Singaporean,1976-02-18,2014-08-08,2014-08-18,0,0.0,0.0,0,NaN,0,1,0,0,0,1,1,1,1,1,1,1,1,14.3,27.2,92.0,76.5,151.0,1,10 days,42.0
2,320aa16c61937447fd6631bf635e7fde,Male,Chinese,Singaporean,1982-07-03,2011-03-10,2011-03-20,0,0.0,0.0,0,0.0,1,0,0,1,1,1,1,0,0,0,1,1,1,14.4,26.6,92.0,69.7,175.0,2,10 days,36.0
3,320aa16c61937447fd6631bf635e7fde,Male,Chinese,Singaporean,1982-07-03,2013-07-02,2013-07-14,0,0.0,0.0,1,NaN,0,0,1,1,0,1,0,1,1,0,0,1,0,18.0,26.6,92.0,66.7,175.0,2,12 days,36.0
4,c7f3881684045e6c49020481020fae36,Male,Malay,Singaporean,1947-06-15,2015-10-28,2015-11-10,0,1.0,0.0,0,0.0,0,0,0,1,0,0,1,1,1,1,1,1,1,14.9,26.7,97.0,96.9,180.0,1,13 days,71.0


In [80]:
patient_data['age'].value_counts().head()

43.0    124
45.0    111
44.0    104
46.0    101
48.0     93
Name: age, dtype: int64

In [74]:
# patient_data.to_csv('clean_datasets/patients_data.csv', sep=',')

Most of the patients are 43 to 46 years old.

---
C3 - Join patients data and the corresponding bill

In [75]:
patients = pd.merge(patient_data, bill, on= ['patient_id', 'date_of_admission'])

In [76]:
patients[patients['patient_id'] == '4e46fddfa404b306809c350aecbf0f6a']

,patient_id,gender,race,resident_status,date_of_birth,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,preop_medication_1,preop_medication_2,preop_medication_3,preop_medication_4,preop_medication_5,preop_medication_6,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height,freq,days_admitted,age,amount
1113,4e46fddfa404b306809c350aecbf0f6a,Male,Chinese,Singaporean,1948-06-16,2011-11-23,2011-12-05,1,1.0,0.0,0,0.0,0,0,1,1,1,0,0,1,1,0,1,1,0,16.1,25.5,111.0,71.4,166.0,4,12 days,70.0,18717.280628
1114,4e46fddfa404b306809c350aecbf0f6a,Male,Chinese,Singaporean,1948-06-16,2013-01-06,2013-01-17,0,0.0,0.0,0,0.0,0,0,0,1,1,1,1,1,1,0,1,0,0,14.1,24.1,103.0,71.4,166.0,4,11 days,70.0,10511.990970
1115,4e46fddfa404b306809c350aecbf0f6a,Male,Chinese,Singaporean,1948-06-16,2013-01-23,2013-02-02,0,0.0,0.0,0,0.0,0,1,0,0,1,1,0,1,0,0,0,0,0,13.0,27.0,91.0,70.4,166.0,4,10 days,70.0,4949.806331
1116,4e46fddfa404b306809c350aecbf0f6a,Male,Chinese,Singaporean,1948-06-16,2013-09-16,2013-09-27,1,1.0,0.0,0,0.0,1,0,0,1,1,0,1,0,1,1,0,0,1,11.7,22.6,84.0,70.4,166.0,4,11 days,70.0,28028.320179


The bill amount can be round to 2 decimal places so that it reflects the price paid by the patients

In [77]:
patients = patients.round({'amount': 2})

In [78]:
patients.head()

,patient_id,gender,race,resident_status,date_of_birth,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,preop_medication_1,preop_medication_2,preop_medication_3,preop_medication_4,preop_medication_5,preop_medication_6,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height,freq,days_admitted,age,amount
0,fa2d818b2261e44e30628ad1ac9cc72c,Female,Indian,Singaporean,1971-05-14,2014-09-20,2014-10-04,0,0.0,0.0,0,0.0,0,1,1,0,1,0,1,1,1,1,0,0,1,14.2,27.0,81.0,68.1,161.0,1,14 days,47.0,19564.17
1,5b6477c5de78d0b138e3b0c18e21d0ae,Female,Chinese,Singaporean,1976-02-18,2014-08-08,2014-08-18,0,0.0,0.0,0,NaN,0,1,0,0,0,1,1,1,1,1,1,1,1,14.3,27.2,92.0,76.5,151.0,1,10 days,42.0,20465.40
2,320aa16c61937447fd6631bf635e7fde,Male,Chinese,Singaporean,1982-07-03,2011-03-10,2011-03-20,0,0.0,0.0,0,0.0,1,0,0,1,1,1,1,0,0,0,1,1,1,14.4,26.6,92.0,69.7,175.0,2,10 days,36.0,14435.91
3,320aa16c61937447fd6631bf635e7fde,Male,Chinese,Singaporean,1982-07-03,2013-07-02,2013-07-14,0,0.0,0.0,1,NaN,0,0,1,1,0,1,0,1,1,0,0,1,0,18.0,26.6,92.0,66.7,175.0,2,12 days,36.0,6578.39
4,c7f3881684045e6c49020481020fae36,Male,Malay,Singaporean,1947-06-15,2015-10-28,2015-11-10,0,1.0,0.0,0,0.0,0,0,0,1,0,0,1,1,1,1,1,1,1,14.9,26.7,97.0,96.9,180.0,1,13 days,71.0,45452.31


In [83]:
# patients.to_csv('clean_datasets/patients.csv')

The `patients` dataset has all the combined information of the costs incurred by a patient for each of the time they were admitted. We have left the NULL values as they are as we cannot impute them. Now we are ready to analyse our data. See you [there](https://github.com/hafsalaeeque/cost-in-healthcare-DS-proj/blob/master/Analysis%20for%20healthcare%20-%20Part%202.ipynb) in part 2!